# Table of Contents
3  ImageCollection概述
4  ImageCollection operator(filter, select, set/get, calculation, toList)
5  Example 1. Calculate NDVI for Jiangxi and Extract NDWI for poyang Lake between 1982 to 2021;
6  Example 2. Download huge ImageCollection from google earth engine; -> Download Images
7  Example 3. Extract NDVI values of climate stations from 1982 to 2021; -> Spatial analysis
8  Example 4. Supervised Classification -> Images Classification
9  Example 5. Unsupervised Classification (clustering) 

In [15]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
# # 先复制路径再复制文件名
# ee_shp = geemap.shp_to_ee(r'C:\Users\xiong\Documents\geemap\shp\jaingxi_dissolve.shp')
# Map.addLayer(ee_shp, {}, 'ee_shp')
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [16]:
# -------------------------------------------------------
# 2  ImageCollection 去云
#  Calculate NDVI for Jiangxi
# -------------------------------------------------------
year = 2022, 2022
doy = '-01-01', '-12-31'

start_date = str(year[0]) + doy[0]
end_date = str(year[1]) + doy[1]

# 需要将数据转为ee.feature的数据类型
jx_bound = ee.FeatureCollection(geemap.shp_to_ee(r'C:/Users/xiong/Documents/geemap/shp/jaingxi_dissolve.shp'))
# print(jx_bound.getInfo())

In [17]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start",'system:id'])

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start",'system:id'])

def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).divide(10000)
    return image.updateMask(mask).copyProperties(image, ["system:time_start",'system:id'])

In [18]:
# Landsat 3 MSS Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1978-03-05 - 1983-03-31
# Landsat 5+7 scenes
# Dataset Availability
# 1984 - 2012
# get landsat 5
collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
# print('Landsat5')
# print(collection5.size().getInfo())
# Landsat 7 Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1999 - 2021
# get landsat 7
# https://developers.google.com/earth-engine/datasets/catalog/landsat
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])

# print(collection5.size().getInfo())

In [5]:
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
col7_ids = collection7.aggregate_array('system:id').getInfo()
# print(col7_ids)

In [6]:
# Landsat 8 Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 2013 - 2021
# get landsat 8
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(maskL8sr).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']) 
print(collection8.size().getInfo())
# Sentinel-2 (S2) Multispectral Instrument (MSI) 
# Dataset Availability
# Level-1C orthorectified top-of-atmosphere reflectance -> 2015-06-23 - present
# Level-2A orthorectified atmospherically corrected surface reflectance.
# get sentinel-2A
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(start_date, end_date).map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) 
# print(sentinel2.size().getInfo())

0


In [7]:
# Map.setCenter(29.50, 116,zoom=4)
sentinel2b = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate('2020-01-01', '2020-12-01').map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) 
Map.addLayer(sentinel2b.median().clip(jx_bound),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},'ly2020')
Map.addLayer(sentinel2.max().clip(jx_bound),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},'ly2022')
Map.split_map(left_layer='ly2020', right_layer='ly2022')

In [8]:
# -------------------------------------------------------
# 3  ImageCollection 
#  Calculate NDVI for Jiangxi
# Bands math operator by image expression
# https://code.earthengine.google.com/2dd2d4e898e633ea5c0c78066f74d1b7?accept_repo=users%2Fnclinton%2FEE101
# -------------------------------------------------------
# function for EVI,GCVI,NDVI,NDWI,LSWI
def f_EVI(img):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B4'),
          'RED': img.select('B3'),
          'BLUE':img.select('B1')})
    return EVI
def f_GCVI(img):
    GCVI = img.expression(
        '(NIR / Green - 1)', {
          'NIR': img.select('B4'),
          'Green': img.select('B2')})
    return GCVI
def addVIs(img):
    NDVI = img.normalizedDifference(['B4','B3']).rename('NDVI')
    mNDWI = img.normalizedDifference(['B2','B5']).rename('mNDWI')
    LSWI = img.normalizedDifference(['B5','B6']).rename('LSWI')
    EVI = f_EVI(img)
    GCVI = f_GCVI(img)
    return img.addBands(NDVI).addBands(NDWI).addBands(LSWI).addBands(EVI).addBands(GCVI)

In [9]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(start_date, end_date).map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']).map(addVIs)

NameError: name 'NDWI' is not defined

In [ ]:
# https://code.earthengine.google.com/c41d27de2bf046ef2cccc75019816497?accept_repo=users%2Fnclinton%2FEE101
# https://github.com/HuiYeAmazing/Study-notes/blob/master/python/jupyter/geemap/Image/03_Image%E5%8F%AF%E8%A7%86%E5%8C%96%E5%8F%82%E6%95%B0%E4%BA%86%E8%A7%A3.md
# ---------------------------
# CSS stands for colors
# https://www.quackit.com/css/tutorial/css_introduction.cfm
# https://www.quackit.com/css/color/charts/css_color_names_chart.cfm
# -----------------------------
ndvi = sentinel2.first().select('NDVI')
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(ndvi, {min: 0, max: 1, palette: palette}, 'NDVI');

In [ ]:
# --*--*----*--*----*--*----*--*----*--*----*--*--
# Landsat8 vs. sentinel2
# comparison for different ImageCollection
# --*--*----*--*----*--*----*--*----*--*----*--*--

# // Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')

# // Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# // Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# # // Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
Map.setOptions('SATELLITE')
Map.addLayer(bare.updateMask(bare), {}, 'bare')